In [20]:
import gpxpy
import gpxpy.gpx
from glob import glob
import os
import datetime
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np
import seawater as sw
import mplleaflet

%matplotlib inline

In [51]:
# This function will loop through and pull in all of the GPX files, accounting for activity type.
def load_data(gpx_path, filter=""):
    gpx_files = glob(os.path.join(gpx_path, filter + "*.gpx"))
    activity_data = []
    counter = 0 # Use to index each new activity to plot each activity in layers. 
    for file_idx, gpx_file in enumerate(gpx_files): 
        gpx = gpxpy.parse(open(gpx_file, 'r'))
        counter = counter + 1
        # Loop through tracks
        for track_idx, track in enumerate(gpx.tracks):
            track_name = track.name
            track_time = track.get_time_bounds().start_time
            track_length = track.length_3d()
            track_duration = track.get_duration()
            track_speed = track.get_moving_data().max_speed
            
            for seg_idx, segment in enumerate(track.segments):
                segment_length = segment.length_3d()
                for point_idx, point in enumerate(segment.points):
                    activity_data.append([counter, os.path.basename(gpx_file), track_idx, track_name, 
                                     track_time, track_length, track_duration, track_speed, 
                                     seg_idx, segment_length, point.time, point.latitude, 
                                     point.longitude, point.elevation, segment.get_speed(point_idx)])
    return activity_data

# This function forces the .gpx files into a DataFrame
def df_activity(activity):
    df = DataFrame(activity, columns=['File_Index', 'File_Name', 'Index', 'Name',
                              'Time', 'Length', 'Duration', 'Max_Speed',
                              'Segment_Index', 'Segment_Length', 'Point_Time', 'Point_Latitude',
                              'Point_Longitude', 'Point_Elevation', 'Point_Speed'])
    return df

# Clean up the gpx DataFrames
def clean_df(df, cols):
    tracks = df[cols].copy()
    tracks['Length'] /= 1e3
    tracks.drop_duplicates(inplace=True)
    return tracks

# Actually set up the plot.
def plot_activity(ax, df, seg_count, color="#000000"):
    for idx in seg_count:
        locs = df[df['File_Index'] == idx].index.tolist()
        ax.plot(df['Point_Longitude'][locs], df['Point_Latitude'][locs],
            color=color, linewidth=2, alpha=1)

In [17]:
# I bulk downloaded my activities (that were valid .gpx files) from my Strava settings and used shell 
# globbing to move them to appropriate folders under ./dat.
run_data  = load_data(gpx_path="./dat/run/", filter="")
ride_data = load_data(gpx_path="./dat/ride/", filter="")
hike_data = load_data(gpx_path="./dat/hike/", filter="")
ski_data  = load_data(gpx_path="./dat/ski/", filter="")
snow_data = load_data(gpx_path="./dat/snowshoe/", filter="")

# Push the .gpx layers into a DataFrame for easy plotting.
df_run    = df_activity(run_data)
df_ride   = df_activity(ride_data)
df_hike   = df_activity(hike_data)
df_ski    = df_activity(ski_data)
df_snow   = df_activity(snow_data)

In [18]:
cols = ['File_Index', 'Time', 'Length', 'Duration', 'Max_Speed']

tracks_run  = clean_df(df_run, cols)
tracks_ride = clean_df(df_ride, cols)
tracks_hike = clean_df(df_hike, cols)
tracks_ski  = clean_df(df_ski, cols)
tracks_snow = clean_df(df_snow, cols)

In [19]:
# Find maximum value for the DataFrame and create an enumerated list of numbers 1 -> max, looping through this
# to individually lay down routes.
numSegs = df_run['File_Index'].max()
seg_count_run = np.arange(1,numSegs + 1) # Second number is not inclusive

numSegs = df_ride['File_Index'].max()
seg_count_ride = np.arange(1,numSegs + 1) 

numSegs = df_hike['File_Index'].max()
seg_count_hike = np.arange(1,numSegs + 1) 

numSegs = df_ski['File_Index'].max()
seg_count_ski = np.arange(1,numSegs + 1) 

numSegs = df_snow['File_Index'].max()
seg_count_snow = np.arange(1,numSegs + 1) 

In [54]:
fig, ax = plt.subplots()

# Just drop columns with NaN... usually only the first.
df_run  = df_run.dropna() 
df_ride = df_ride.dropna()
df_hike = df_hike.dropna()
df_ski  = df_ski.dropna()
df_snow = df_snow.dropna()

plot_activity(ax, df_run, seg_count_run, color="#fbb4ae")
plot_activity(ax, df_ride, seg_count_ride, color="#decbe4")
plot_activity(ax, df_hike, seg_count_hike, color="#e5d8bd")
plot_activity(ax, df_ski, seg_count_ski, color="#fed9a6")
plot_activity(ax, df_snow, seg_count_snow, color="#fed9a6")

# Can go into _map.html that is rendered and adjust the javascript to change e.g. default zoom. Look at D3/JS tutorials 
# using mplleaflet. For example, do map.setView([40.02, -105.27], 13) to default to Boulder.
mplleaflet.show(fig=fig, tiles="cartodb_positron") 
